In [ ]:

OUTPUT_FILE = "output_yeast_structure.csv"
INPUT_FILE = '/home/imhof_team/Public/mauricio/workflow/yeast_type3/yeast_type3_safe2.fasta'

import logging
logging.basicConfig(level=logging.DEBUG, format='%(levelname)s - %(message)s')

#import experiments
import seqdhbm.SeqDHBM as SeqDHBM
import seqdhbm.fasta as fasta
import seqdhbm.pdbfiles as pdbfiles
import sys, os
import shutil
import time

def workflow(jobnum= "J0", fastafile=None, pdbs=[], pdbid="", rawseq="", mode="structure"):
    """ Runs the workflow 
    
    Params:
    - fasta: One fasta file
    - pdbfiles: list of pdb filenames, they should be in the root dir
    - pdbid: string with PDB codes (comma separated)
    - knowstruc: boolean: we use services to obtain the structure if False"""
    # TODO: use jobnum to organize submissions
    # Deal with the fasta file with multiple sequences
    seq_dict = {}
    
    seq_list = []
    if fastafile:
        """seq_dict = fasta.fasta_to_seq(fastafile)
        fasta.organize_sequences(seq_dict)"""
        seq_list = fasta.fasta_to_seq2(fastafile)
        fasta.organize_sequences2(seq_dict)
        #logging.debug("fasta:")
        #logging.debug(seq_dict.keys())

    # Deal with the pdb ids (by downloading them)
    pdbid_dict = {} # stores the pdbfile path to each pdb id
    if pdbid:
        raise Exception("not prepared to work with they new variables")
        l = pdbfiles.text_to_list(pdbid)
        pdbid_dict = pdbfiles.get_pdb_files(l)
        #logging.debug("pdbid")
        #logging.debug(pdbid_dict)
    
    if rawseq:
        seq_dict["Your input sequence"] = ("Your input sequence header", rawseq)
        folder = "%s/MI/"%jobnum
        file = folder+"MI1.fasta"
        name = "Your input sequence"
        assert False, "create the folder first"
        with open(file, "w") as f:
            f.write(">%s\n"%name)
            lines=[rawseq[x:x+60] for x in range(0, len(rawseq), 60)]
            f.write('\n'.join(lines))
        seq_list += [{"seq":rawseq, 
                     "name":name, 
                     "folder": folder,
                     "file": file,
                     "submited_as": "Manual input"}]
    
    # Arrange the pdb files (user-submitted or downloaded)
    ## Put the structure file in folders - for docking and MD
    ## get the primary sequence for motif search
    for file in pdbs:
        raise Exception("not prepared to work with they new variables")
        try:
            folder = ".".join(file.split(".")[:-1]) # remove extension
        except:
            folder = file
        try:
            os.makedirs(folder, exist_ok=True)
            shutil.move(file, folder+"/"+file)
            pdbid_dict[folder] = folder+"/"+file
        except:
            logging.error("Could not move file"+ file + "\nSkiping it!") # TODO: List of warnings and errors
            # TODO: organize the files into folders
    
    # TODO: convert the pdbs into fasta
    for file in pdbid_dict.values():
        seq = pdbfiles.structure_to_fasta(file)
        if seq:
            seq_dict[file] = (">header", seq)
    # Run the motif check
    #logging.debug("*"*30)
    #logging.debug(seq_dict)
    
    #spacerinfo = {}
    
    hbm_result = {}
    cnt_progress = 0
    seq_dict = {}
    for item in seq_list:
        cnt_progress+=1
        try:
            coordsites = SeqDHBM.SpotCoordinationSite({">"+item["name"]: item["seq"]}, mode)
            for fold, coordsite in coordsites.items():
                # TODO: Handle what is to be done with the coordination sites
                # Output? Save in our user output?
                hbm_result[fold[1:]] = coordsite
                item["result"] = coordsite["result"]
                
        except Exception as e:
            logging.error("****")
            logging.error("%s %s"%(folder, header))
            logging.error(e)
            logging.error(type(e))
            logging.error(e.args)
            logging.error("****")
        finally:
            logging.debug("Got the results for %s"%folder)
            logging.debug("%d out of %d complete"%(cnt_progress, len(seq_dict)))

    return seq_list

    for folder, (header, seq) in seq_dict.items():
        """fo = "Saved_run/IL-1A(271)/IL-1A(271).fasta"
    
        SeqDHBM.ReadFasta(fo)"""
        #print(folder, "$", seq)
        cnt_progress+=1
        try:
            coordsites = SeqDHBM.SpotCoordinationSite({">"+folder: seq}, mode)
            for fold, coordsite in coordsites.items():
                # TODO: Handle what is to be done with the coordination sites
                # Output? Save in our user output?
                hbm_result[fold[1:]] = coordsite
        except Exception as e:
            logging.error("****")
            logging.error("%s %s"%(folder, header))
            logging.error(e)
            logging.error(type(e))
            logging.error(e.args)
            logging.error("****")
        finally:
            logging.debug("Got the results for %s"%folder)
            logging.debug("%d out of %d complete"%(cnt_progress, len(seq_dict)))
    if hbm_result:
        print("*"*60)
        print("Results for the HBM analysis")
        
    """with open(OUTPUT_FILE, "w") as f:
        f.write("Name\tCoordinating residue\tSpacer > 2\n")
        for fold, sites in hbm_result.items():
            print("Analysis of the sequence %s"%fold[1:])
            for pos, spac in sites.items():
                print("%s - %s"%(str.rjust(pos,6), spac))
                f.write("%s\t%s\t%s\n"%(fold, pos, spac))
                """; # Maybe i don't need that csv output
    return hbm_result


if (__name__ == "__main__"):

    fastafile = ''
    mode = ""
    if sys.argv[0].endswith("ipykernel_launcher.py"):
        fastafile = INPUT_FILE
        mode = "wesa"
        scriptname = "workflow" # TODO: Will this be the final name?
    else:
        scriptname = os.path.basename(__file__)
    pdbids = ""# "1S0L,1L2H,1I1B,1HIB,1I8H"
    pdbs=[]
    
    for params in sys.argv:
        if params.startswith("-pdbid="):
            pdbids = params[7:]
        elif params.startswith("-fasta="):
            fastafile = params[7:]
        elif params.startswith("-pdbfiles="):
            # param pdbid must be formatted as list
            pdbs = eval(params[10:])
        elif params.startswith("-mode="):
            # param pdbid must be formatted as list
            mode = params[6:].lower()
        elif params.startswith("-help"):
            print("Usage Help")
            print("call by using 'python %s -mode=<mode> -fasta=<fastafile> -pdbid=<pdb_id> -pdbfiles=<pdb_files>'"%(scriptname))
            print("<mode> is mandatory and accepts:")
            print("-mode=structure: if structure is known (default)")
            print("-mode=wesa: to use our services to PREDICT surface accessibility")
            print("<fastafile> is the full path of a fasta file with one or more sequences")
            print("<pdb_id> must contain the PDB ids, separated by commas.")
            print("<pdb_files> PDB files with .ent or .pdb format")
    
    assert mode in ["structure", "wesa"], ("Mode not detected!\n"
    "Use 'python %s' for help on using the application\n")
            
    workflow(jobnum=0, fastafile=fastafile, pdbs=pdbs, pdbid=pdbids, mode=mode)



INFO - --------------------------
INFO - SEQUENCE VALIDITY CHECK:
INFO - --------------------------
INFO - NOTE : All the 1 sequences were verified to be valid !
INFO - --------------------------------------------------------------------------------
INFO - --------------------------------------------------------------------------------
INFO - WORKING ON THE VALID SEQUENCE 1 . SEC31(1273)
INFO - --------------------------------------------------------------------------------
INFO - MVKLAEFSRTATFAWSHDKIPLLVSGTVSGTVDANFSTDSSLELWSLLAADSEKPIASLQVDSKFNDLDWSHNNKIIAGALDNGSLELYSTNEANNAINSMARFSNHSSSVKTVKFNAKQDNVLASGGNNGEIFIWDMNKCTESPSNYTPLTPGQSMSSVDEVISLAWNQSLAHVFASAGSSNFASIWDLKAKKEVIHLSYTSPNSGIKQQLSVVEWHPKNSTRVATATGSDNDPSILIWDLRNANTPLQTLNQGHQKGILSLDWCHQDEHLLLSSGRDNTVLLWNPESAEQLSQFPARGNWCFKTKFAPEAPDLFACASFDNKIEVQTLQNLTNTLDEQETETKQQESETDFWNNVSREESKEKPTVFHLQAPTWYGEPSPAAHWAFGGKLVQITPDGKGVSITNPKISGLESNTTLSEALKTKDFKPLINQRLVKVIDDVNEEDWNLLEKLSMDGTEEFLKEALAFDNDESDAQDDANNEKEDDGEEFFQQIETNFQPEGDFSLSGNIEQTI

A4
C10


INFO - ********************************************************************************
INFO - NOTE : Your sequence has been posted to the WESA server for solvent accessibility prediction
INFO - NOTE : This might take a few minutes....grab a coffee maybe?
INFO - NOTE : Making attempts at 30 second intervals to see if the WESA prediction is complete
INFO - ********************************************************************************
INFO - NOTE : Attempt 1 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 2 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 3 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 4 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Att

INFO - NOTE : Attempt 58 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 59 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 60 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 61 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 62 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 63 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 64 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 65 to fetch WESA ou

INFO - NOTE : Attempt 118 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 119 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 120 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 121 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 122 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 123 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 124 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 125 to fetch

INFO - NOTE : Attempt 6 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 7 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 8 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 9 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 10 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 11 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 12 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 13 to fetch WESA output

INFO - NOTE : Attempt 16 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 17 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 18 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 19 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 20 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 21 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 22 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 23 to fetch WESA ou

INFO - NOTE : Attempt 1 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 2 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 3 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 4 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 5 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 6 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 7 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 8 to fetch WESA output...


INFO - NOTE : Attempt 61 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 62 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 63 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 64 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 65 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 66 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 67 to fetch WESA output...
INFO - ********************************************************************************
INFO - NOTE : Attempt 68 to fetch WESA ou